***USE nyc_Jan_Jun_2016_weat.csv DATSET ITS UPLOADED ON GITHUB***

In [1]:
# Install required packages
!pip install pandas folium geopy

In [2]:
import pandas as pd
import folium
from folium.plugins import HeatMap
from geopy.geocoders import Nominatim
from google.colab import files

In [3]:
# Upload CSV file
uploaded = files.upload()

Saving nyc_Jan_Jun_2016_weat.csv to nyc_Jan_Jun_2016_weat.csv


In [4]:
# Read the CSV file
data = pd.read_csv("nyc_Jan_Jun_2016_weat.csv")

# Display the first few rows of the dataset
data.head()

,DATE,LATITUDE,LONGITUDE,SNOW,PRCP,SNWD,TAVG,TMAX,TMIN
0,20160101,41.0639,-73.7722,0.0,0.0,-9999.0,-9999,-9999,-9999
1,20160102,41.0639,-73.7722,0.0,0.0,-9999.0,-9999,-9999,-9999
2,20160103,41.0639,-73.7722,0.0,0.0,-9999.0,-9999,-9999,-9999
3,20160104,41.0639,-73.7722,0.0,0.0,-9999.0,-9999,-9999,-9999
4,20160105,41.0639,-73.7722,0.0,0.0,-9999.0,-9999,-9999,-9999


In [5]:
# Convert the SNOW column to numeric
data["SNOW"] = pd.to_numeric(data["SNOW"], errors='coerce')

# Filter data for positive snow values
snow_data_positive = data[data["SNOW"] > 0]

In [6]:
# Get maximum snow depth for each location
max_snow_per_location = snow_data_positive.loc[snow_data_positive.groupby(['LATITUDE', 'LONGITUDE'])['SNOW'].idxmax()]
print("\nSnow Depth in Inches for Each Location:")
for index, row in max_snow_per_location.iterrows():
    print(f"Latitude: {row['LATITUDE']}, Longitude: {row['LONGITUDE']}, Snow Depth: {row['SNOW']:.2f}")


Snow Depth in Inches for Each Location:
Latitude: 40.2754, Longitude: -74.006, Snow Depth: 10.00
Latitude: 40.297, Longitude: -74.0015, Snow Depth: 11.00
Latitude: 40.3204, Longitude: -74.1927, Snow Depth: 11.50
Latitude: 40.3622, Longitude: -74.0833, Snow Depth: 12.00
Latitude: 40.3671, Longitude: -73.9979, Snow Depth: 15.50
Latitude: 40.4057, Longitude: -74.2549, Snow Depth: 3.80
Latitude: 40.4562, Longitude: -74.2413, Snow Depth: 13.00
Latitude: 40.4739, Longitude: -74.4, Snow Depth: 1.70
Latitude: 40.5421, Longitude: -74.1476, Snow Depth: 5.00
Latitude: 40.5554, Longitude: -74.274, Snow Depth: 12.00
Latitude: 40.5674, Longitude: -74.1327, Snow Depth: 19.50
Latitude: 40.5759, Longitude: -74.2847, Snow Depth: 20.10
Latitude: 40.6386, Longitude: -73.7622, Snow Depth: 8.30
Latitude: 40.63861, Longitude: -73.76222, Snow Depth: 30.30
Latitude: 40.6588, Longitude: -74.3358, Snow Depth: 8.50
Latitude: 40.6623, Longitude: -73.678, Snow Depth: 17.30
Latitude: 40.66694, Longitude: -74.32306,

In [7]:
# Initialize geolocator
geolocator = Nominatim(user_agent="snow_depth_locator")
top_3_snow_locations = max_snow_per_location.nlargest(3, 'SNOW')
print("\nTop 3 Locations with Highest Snow Depth:")
for index, row in top_3_snow_locations.iterrows():
    location = geolocator.reverse((row['LATITUDE'], row['LONGITUDE']), exactly_one=True)
    address = location.address if location else "Address not found"
    print(f"Latitude: {row['LATITUDE']}, Longitude: {row['LONGITUDE']}, Address: {address}, Snow Depth: {row['SNOW']:.2f}")


Top 3 Locations with Highest Snow Depth:
Latitude: 40.85, Longitude: -74.06139, Address: Teterboro Airport, Poole Street, Moonachie, Bergen County, New Jersey, 07072, United States, Snow Depth: 68.00
Latitude: 40.63861, Longitude: -73.76222, Address: JFK Access Road, Queens, Queens County, City of New York, New York, 11430, United States, Snow Depth: 30.30
Latitude: 40.77944, Longitude: -73.88028, Address: Bowery Bay Boulevard, Queens, Queens County, City of New York, New York, 11371, United States, Snow Depth: 27.90


In [9]:
# Prepare data for heatmap
heat_data = max_snow_per_location[['LATITUDE', 'LONGITUDE', 'SNOW']].values.tolist()
map_center = [40.7128, -74.0060]
map_nyc = folium.Map(location=map_center, zoom_start=11)
HeatMap(heat_data, radius=45).add_to(map_nyc)

# Save the map as HTML
map_nyc.save("snowcover_map.html")
map_nyc